# working model for tensorfusion

In [1]:
import pickle
import torch
from torchmetrics import F1Score
import pandas as pd
import numpy as np
from glob import glob
from collections import Counter
import torch.nn.functional as F

print(torch.__version__)

/home/arplab/project/lou/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1+cu117


## wav, text데이터 불러오기

In [2]:
# wav and text data load
dataset_file_lst = glob('../model/data/lou_dataset*.pkl')
dataset_file_lst = sorted(dataset_file_lst)
raw_dataset = {}
for i in dataset_file_lst:
    with open(i, 'rb') as f:
        new_dataset = pickle.load(f)
        raw_dataset  = dict(**raw_dataset, **new_dataset)
session = 'Session01'
print(raw_dataset.keys(),'\n', raw_dataset[session].keys())

dict_keys(['Session11', 'Session12', 'Session13', 'Session01', 'Session02', 'Session03', 'Session04', 'Session05', 'Session06', 'Session07', 'Session08', 'Session09', 'Session10']) 
 dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])


In [3]:
annot_lst = glob('../org_KEMDy20/annotation/*.csv')
emotion_list = []
for annot_file in annot_lst:
    annot = pd.read_csv(annot_file, skiprows=1)
    emotion_list.append(annot['Emotion'])
emotion_list = list(pd.Series([j for i in emotion_list for j in i]).unique())
len(emotion_list)


24

In [4]:
emotion_lst = []
sessions = sorted(list(raw_dataset.keys()))
sessions.remove('Session07')
sessions.remove('Session12')

for session in sessions:
    emotion_lst.append(raw_dataset[session]['Emotion'].unique())

emotion_lst = [j for i in emotion_lst for j in i]
emotion_lst = list(pd.Series(emotion_lst).unique())
len(emotion_lst)


20

<2020년>
'angry;disqust',  'angry;disqust;fear;neutral;sad',  'angry;disqust;neutral', 'angry;happy;neutral', 'angry;neutral', 'disqust;happy;neutral', 'disqust;neutral', 'disqust;neutral;sad', 'fear;happy', 'fear;happy;neutral', 'fear;neutral', 'happy;neutral', 'happy;neutral;surprise', 'happy;sad', 'happy;surprise', 'neutral;sad', 'neutral;surprise'

<2019년>
'angry;disgust;fear;neutral;surprise', 'angry;fear', 'angry;fear;neutral', 'angry;fear;surprise', 'angry;happy', 'angry;neutral;surprise', 'angry;sad', 'angry;surprise', 'disgust;fear', 'disgust;happy','disgust;neutral;surprise', 'disgust;sad', 'disgust;surprise', 'fear;neutral;surprise', 'fear;sad', 'fear;surprise', 'happy;neutral;sad', 'neutral;sad;surprise', 'sad;surprise'}


In [5]:
encode_dict = {'angry':0, 'disgust':1, 'fear':2,'happy':3,'neutral':4, 'sad':5, 'surprise':6,  
               'neutral;surprise': 450, 'neutral;sad': 460, 'happy;neutral': 340, 
               'angry;neutral': 40, 'disgust;neutral': 140, 'fear;neutral': 240, 
               'happy;surprise': 350, 'angry;happy;neutral': 7340, 'angry;disgust': 10, 
               'happy;neutral;surprise': 3450, 'fear;happy': 230,'fear;happy;neutral': 2340,
               'angry;disgust;neutral': 7140, 'disgust;neutral;sad': 1460, 
               'happy;sad': 360, 'disgust;happy;neutral': 3410, 'angry;fear': 20, 'angry;fear;neutral':7240,
               'angry;fear;surprise': 7250, 'angry;happy': 730, 'angry;neutral;surprise':7450, 
               'angry;sad': 60, 'angry;surprise': 50, 'disgust;fear':120, 'disgust;happy': 130,
               'disgust;neutral;surprise':1450, 'disgust;sad': 160, 'disgust;surprise':150, 
               'fear;neutral;surprise':2450, 'fear;sad': 260, 'fear;surprise':250, 'happy;neutral;sad':3460,
               'neutral;sad;surprise':4560, 'sad;surprise': 560,
               'angry;disgust;fear;neutral;sad': 10000,'angry;disgust;fear;neutral;surprise':20000}

In [6]:
encode_dict = {k: v for k, v in sorted(encode_dict.items(), key=lambda item: item[1])}
encode_dict

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6,
 'angry;disgust': 10,
 'angry;fear': 20,
 'angry;neutral': 40,
 'angry;surprise': 50,
 'angry;sad': 60,
 'disgust;fear': 120,
 'disgust;happy': 130,
 'disgust;neutral': 140,
 'disgust;surprise': 150,
 'disgust;sad': 160,
 'fear;happy': 230,
 'fear;neutral': 240,
 'fear;surprise': 250,
 'fear;sad': 260,
 'happy;neutral': 340,
 'happy;surprise': 350,
 'happy;sad': 360,
 'neutral;surprise': 450,
 'neutral;sad': 460,
 'sad;surprise': 560,
 'angry;happy': 730,
 'disgust;neutral;surprise': 1450,
 'disgust;neutral;sad': 1460,
 'fear;happy;neutral': 2340,
 'fear;neutral;surprise': 2450,
 'disgust;happy;neutral': 3410,
 'happy;neutral;surprise': 3450,
 'happy;neutral;sad': 3460,
 'neutral;sad;surprise': 4560,
 'angry;disgust;neutral': 7140,
 'angry;fear;neutral': 7240,
 'angry;fear;surprise': 7250,
 'angry;happy;neutral': 7340,
 'angry;neutral;surprise': 7450,
 'angry;disgust;fear;neutral;sad': 10000,

In [7]:
# encoding Emotion for whole data
# 사전에 실제로 encoding한 끝 수가 마지막 linear layer의 끝자리랑 맞아야 합니다. 아니면 CUDA error: CUBLAS_STATUS_EXECUTION_FAILED가 나는 것 같아요.
# 예를 들어, label이 0~9, 11,13이렇게 12개가 되었어도, 0~13은 14개니까 마지막 레이어에서 14개 unit을 받아야 multiclass classification이 에러없이 진행됩니다!
# 데이터에서 정답 라벨 인코딩: ['neutral', 'happy', 'surprise', 'disgust', 'angry', 'sad', 'fear']
# 이 순서를 지켜서 라벨링을 해야함
decode_dict = {b:i for i, b in encode_dict.items()}
encode_dict, decode_dict

({'angry': 0,
  'disgust': 1,
  'fear': 2,
  'happy': 3,
  'neutral': 4,
  'sad': 5,
  'surprise': 6,
  'angry;disgust': 10,
  'angry;fear': 20,
  'angry;neutral': 40,
  'angry;surprise': 50,
  'angry;sad': 60,
  'disgust;fear': 120,
  'disgust;happy': 130,
  'disgust;neutral': 140,
  'disgust;surprise': 150,
  'disgust;sad': 160,
  'fear;happy': 230,
  'fear;neutral': 240,
  'fear;surprise': 250,
  'fear;sad': 260,
  'happy;neutral': 340,
  'happy;surprise': 350,
  'happy;sad': 360,
  'neutral;surprise': 450,
  'neutral;sad': 460,
  'sad;surprise': 560,
  'angry;happy': 730,
  'disgust;neutral;surprise': 1450,
  'disgust;neutral;sad': 1460,
  'fear;happy;neutral': 2340,
  'fear;neutral;surprise': 2450,
  'disgust;happy;neutral': 3410,
  'happy;neutral;surprise': 3450,
  'happy;neutral;sad': 3460,
  'neutral;sad;surprise': 4560,
  'angry;disgust;neutral': 7140,
  'angry;fear;neutral': 7240,
  'angry;fear;surprise': 7250,
  'angry;happy;neutral': 7340,
  'angry;neutral;surprise': 7450,


In [8]:
sessions

['Session01',
 'Session02',
 'Session03',
 'Session04',
 'Session05',
 'Session06',
 'Session08',
 'Session09',
 'Session10',
 'Session11',
 'Session13']

In [9]:
merged_dataset = {}
for session_key in sessions: # session에는 이미 session07과 12가 제외되어 있음
    raw_dataset[session_key]['Emotion'] = raw_dataset[session_key]['Emotion'].map(encode_dict) #encoding
    
    for data_name in raw_dataset[session_key].keys():
        if data_name in merged_dataset.keys():
            for data in raw_dataset[session_key][data_name]:
                merged_dataset[data_name].append(data)
        else:
            merged_dataset[data_name] = []
            for data in raw_dataset[session_key][data_name]:
                merged_dataset[data_name].append(data)
    
for data_name in merged_dataset.keys():
    if data_name == 'text_embeddings' or data_name == 'wav_embeddings':
        merged_dataset[data_name] = torch.stack(merged_dataset[data_name])

merged_dataset.keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])

In [10]:
# check the length of each data column
for i in merged_dataset.keys():
    print(f"{i}: ", len(merged_dataset[i]))

file_names:  3725
text_embeddings:  3725
wav_embeddings:  3725
Emotion:  3725
Arousal:  3725
Valence:  3725


## Temp, EDA data 불러오기

In [11]:
# data load
# 아래 데이터에서 new_emotion의 라벨링 의미는 순서대로 ['neutral', 'happy', 'surprise', 'disgust', 'angry', 'sad', 'fear']
with open('../model/data/lou_ts_dataset.pkl', 'rb') as f:
    ts_dataset = pickle.load(f)
    # TS데이터가 없는 세션 drop
    ts_dataset = ts_dataset[~ts_dataset['segment_id'].str.startswith('Sess12')] 
    ts_dataset = ts_dataset[~ts_dataset['segment_id'].str.startswith('Sess17')]
print(len(ts_dataset))
ts_dataset.sample(3)
ts_dataset['Emotion'] = ts_dataset['Emotion'].str.replace('disqust', 'disgust')
ts_dataset['Emotion'] = ts_dataset['Emotion'].apply(lambda x: ';'.join(sorted(x.split(';'))))
ts_dataset = ts_dataset.reset_index(drop=True)
ts_dataset

12786


,,start,end,segment_id,Emotion,Valence,Arousal,Emotion.1,Valence.1,Arousal.1,...,Emotion.9,Valence.9,Arousal.9,Emotion.10,Valence.10,Arousal.10,index,eda,temp,new_emotion
0,1,3.578390,11.637391,Sess01_script01_User002M_001,neutral,3.4,2.9,neutral,3,3,...,neutral,3,3,neutral,3,3,0,"[-0.06791500000000017, -0.110201, -0.025627999...","[34.81, 34.81, 34.81, 34.79, 34.79, 34.79, 34....","[10, 0, 0, 0, 0, 0, 0]"
1,2,11.637391,23.334393,Sess01_script01_User002M_002,neutral,3.1,2.9,neutral,3,3,...,neutral,3,3,neutral,3,3,1,"[0.006407000000000274, -0.021784000000000248, ...","[34.79, 34.79, 34.79, 34.77, 34.77, 34.77, 34....","[10, 0, 0, 0, 0, 0, 0]"
2,3,23.334393,31.558392,Sess01_script01_User002M_003,neutral,3.1,3.0,neutral,3,3,...,neutral,3,3,neutral,3,3,2,"[0.07944700000000005, 0.05125599999999997, 0.0...","[34.75, 34.75, 34.75, 34.75, 34.79, 34.79, 34....","[10, 0, 0, 0, 0, 0, 0]"
3,4,31.558392,42.970390,Sess01_script01_User002M_004,neutral,3.7,3.1,neutral,4,3,...,neutral,3,3,neutral,4,3,3,"[0.0038439999999999586, -0.03331699999999982, ...","[34.77, 34.77, 34.77, 34.77, 34.77, 34.77, 34....","[9, 1, 0, 0, 0, 0, 0]"
4,5,42.777290,44.863290,Sess01_script01_User001F_001,neutral,3.8,2.8,happy,5,2,...,neutral,3,3,happy,4,3,4,"[0.1241910000000006, 0.09474499999999964, 0.03...","[34.55, 34.55, 34.55, 34.55, 34.55, 34.55, 34....","[6, 4, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12781,494,289.449980,294.943000,Sess40_script06_User079F_043,neutral,3.0,3.5,neutral,3,4,...,neutral,3,4,neutral,3,4,493,"[-0.023063999999999973, -0.02050200000000002, ...","[35.07, 35.07, 35.07, 35.07, 35.05, 35.05, 35....","[10, 0, 0, 0, 0, 0, 0]"
12782,495,294.943000,297.598000,Sess40_script06_User079F_044,neutral,2.9,3.1,neutral,3,2,...,neutral,3,4,neutral,2,4,494,"[-0.0064070000000000515, -0.005125000000000046...","[35.05, 35.05, 35.07, 35.07, 35.07, 35.07, 35....","[10, 0, 0, 0, 0, 0, 0]"
12783,496,297.598000,304.171000,Sess40_script06_User079F_045,neutral,2.6,3.6,neutral,2,3,...,fear,2,4,neutral,3,4,495,"[-0.007688000000000028, -0.002561999999999953,...","[35.05, 35.05, 35.05, 35.05, 35.05, 35.05, 35....","[9, 0, 0, 0, 0, 0, 1]"
12784,497,304.171000,306.863000,Sess40_script06_User079F_046,neutral,2.7,3.3,disgust,2,3,...,neutral,2,4,surprise,3,4,496,"[-0.0064069999999999405, -0.003844000000000069...","[35.05, 35.07, 35.07, 35.07, 35.07, 35.07, 35....","[8, 0, 1, 1, 0, 0, 0]"


In [12]:
# https://stats.stackexchange.com/questions/107874/how-to-deal-with-a-skewed-class-in-binary-classification-having-many-features


# torch dataset 만들기
- 참고: https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html

In [13]:
import os
import pandas as pd
from datasets import Dataset
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split

In [14]:
class EtriDataset(Dataset):
    def __init__(self, file_names, 
                 text_embeddings, 
                 wav_embeddings, 
                 Temp,
                 EDA,
                 Emotion,
                 Emotion_ext, 
                 Arousal, 
                 Valence):
        self.file_names = file_names
        self.text_embeddings = text_embeddings
        self.wav_embeddings = wav_embeddings
        self.temp = Temp
        self.eda = EDA
        self.label_emotion = Emotion
        self.label_emotion_ext = Emotion_ext
        self.label_arousal = Arousal
        self.label_valence = Valence
        
    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        text_embeddings = self.text_embeddings[idx]
        wav_embeddings = self.wav_embeddings[idx]
        temp = self.temp[idx]
        eda = self.eda[idx]
        label_emotion = self.label_emotion[idx]
        label_emotion_ext = self.label_emotion_ext[idx]
        label_arousal = self.label_arousal[idx]
        label_valence = self.label_valence[idx]
        return text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence

In [15]:
# data load 및 나누기: https://076923.github.io/posts/Python-pytorch-11/

# session을 통합시킨 데이터 셋을 만들었을 때
dataset = EtriDataset(file_names = merged_dataset['file_names'],
                      text_embeddings = merged_dataset['text_embeddings'],
                      wav_embeddings = merged_dataset['wav_embeddings'],
                      Emotion = ts_dataset['Emotion'][:3725],
                      Arousal = torch.Tensor(merged_dataset['Arousal']),
                      Valence = torch.Tensor(merged_dataset['Valence']),
                      EDA = torch.Tensor(ts_dataset['eda'][:3725]), ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함
                      Temp = torch.Tensor(ts_dataset['temp'][:3725]), ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함
                      Emotion_ext = torch.Tensor(ts_dataset['new_emotion'][:3725])) ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함


/tmp/ipykernel_2163655/3606781350.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  EDA = torch.Tensor(ts_dataset['eda'][:3725]), ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함


In [16]:
dataset_size = len(dataset)
train_size = int(dataset_size * 0.7)
validation_size = int(dataset_size * 0.15)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

print(train_size, test_size, validation_size)
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

2607 560 558
Training Data Size : 2607
Validation Data Size : 558
Testing Data Size : 560


In [17]:
train_dataloader = DataLoader(train_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=256, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=True, drop_last=True)

# NetWork 만들기

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [19]:
class MLPNetwork_pre(nn.Module):
    def __init__(self, input_length, input_width):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_length*input_width, 768)
        self.gelu1 = nn.GELU()
        self.bn1 = nn.BatchNorm1d(768)
        self.fc2 = nn.Linear(768, 512)
        self.gelu2 = nn.GELU()
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 32)
        self.gelu3 = nn.GELU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.gelu1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.gelu2(x)
        x = self.bn2(x)
        x = self.fc3(x)
        output = self.gelu3(x)
        return output
    
class ConvNetwork_pre(nn.Module):
    def __init__(self, input_channel):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels = input_channel, out_channels= 32, kernel_size = 3, padding = 1)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv1d(in_channels = 32, out_channels = 10, kernel_size = 3, padding = 1)
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        output = self.relu2(x)
        return output

class MLPNetwork_final(nn.Module):
    def __init__(self, input_length, input_width, input_depth):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_length*input_width*input_depth, 256)
        self.gelu1 = nn.GELU()
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 64)
        self.gelu2 = nn.GELU()
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 7)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.gelu1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.gelu2(x)
        x = self.bn2(x)
        output = self.fc3(x)
        
        return output


In [20]:
class TensorFusionMixer(nn.Module):
    def __init__(self, ModelA, ModelB, ModelC, ModelD, ModelE):
        super().__init__()
        self.ModelA = ModelA
        self.ModelB = ModelB
        self.ModelC = ModelC
        self.ModelD = ModelD
        self.Model_mlp_final = ModelE
        self.softmax = nn.Softmax(dim = 1)
        # self.softmax = nn.Softmax(dim=1)
        
    def tensor_fusion(self, batch_arr1, batch_arr2, batch_arr3):
        fusion_matrix_lst = []
        for i, (arr1, arr2, arr3) in enumerate(zip(batch_arr1, batch_arr2, batch_arr3)):
            arr1 = arr1.unsqueeze(-1).unsqueeze(-1)
            arr2 = arr2.unsqueeze(0).unsqueeze(-1)
            arr3 = arr3.squeeze().unsqueeze(0).unsqueeze(0)
            
            # outer_matrix = torch.einsum('i,j,kp->ijk', arr1, arr2, arr3)
            outer_matrix = torch.kron(torch.kron(arr1,arr2), arr3)
            l, w, d = outer_matrix.shape
            
            outer_matrix = outer_matrix.view(-1, l, w, d)
            fusion_matrix_lst.append(outer_matrix)
            
        fusion_matrix = torch.concat(fusion_matrix_lst)
        
        return fusion_matrix
        
    def forward(self, x1, x2, x3, x4):
        x1 = self.ModelA(x1)
        x2 = self.ModelB(x2)
        x3 = self.ModelC(x3)
        x4 = self.ModelD(x4)
        x5 = torch.cat([x3,x4], dim=0)
        
        fusion_matrix = self.tensor_fusion(x1, x2, x5) 
        # x = self.Model_mlp_final(fusion_matrix) # 기존 emotion사용
        x = self.Model_mlp_final(fusion_matrix) # 새로운 emotion사용
        output = self.softmax(x)
        return output




In [21]:
# txt_input_length, txt_input_width = raw_dataset[session]['text_embeddings'][0].shape | 마지막엔 지울 것
# _, wav_input_length, wav_input_width = raw_dataset[session]['wav_embeddings'][0].shape
txt_input_length, txt_input_width = merged_dataset['text_embeddings'][0].shape
_, wav_input_length, wav_input_width = merged_dataset['wav_embeddings'][0].shape
temp_input_length = ts_dataset['temp'][0].shape[0]
eda_input_length = ts_dataset['eda'][0].shape[0]

# tf_mixer에 들어갈 wav mlp, txt mlp 선언
model_mlp_txt = MLPNetwork_pre(txt_input_length,txt_input_width).to(device)
model_mlp_wav = MLPNetwork_pre(wav_input_length,wav_input_width).to(device)
model_conv_temp = ConvNetwork_pre(temp_input_length).to(device)
model_conv_eda = ConvNetwork_pre(eda_input_length).to(device)

model_mlp_final = MLPNetwork_final(32,32,10).to(device)

# 최종 모델 선언
model_tf_mixer = TensorFusionMixer(ModelA = model_mlp_txt, 
                                   ModelB = model_mlp_wav,
                                   ModelC = model_conv_temp,
                                   ModelD = model_conv_eda,
                                   ModelE = model_mlp_final).to(device)

# model 병렬 학습 처리
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_mlp_txt = nn.DataParallel(model_mlp_txt).to(device)
    model_mlp_wav = nn.DataParallel(model_mlp_wav).to(device)
    model_conv_temp = nn.DataParallel(model_conv_temp).to(device)
    model_conv_eda = nn.DataParallel(model_conv_eda).to(device)
    model_tf_mixer = nn.DataParallel(model_tf_mixer).to(device)
print(model_tf_mixer)

Let's use 4 GPUs!
DataParallel(
  (module): TensorFusionMixer(
    (ModelA): MLPNetwork_pre(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (fc1): Linear(in_features=61440, out_features=768, bias=True)
      (gelu1): GELU(approximate='none')
      (bn1): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=768, out_features=512, bias=True)
      (gelu2): GELU(approximate='none')
      (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc3): Linear(in_features=512, out_features=32, bias=True)
      (gelu3): GELU(approximate='none')
    )
    (ModelB): MLPNetwork_pre(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (fc1): Linear(in_features=50176, out_features=768, bias=True)
      (gelu1): GELU(approximate='none')
      (bn1): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=768, out_features=512, bias=Tr

# 학습을 위한 train, test method 만들기

In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)    
    # data 순서: text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence
    for batch, (X_txt, X_wav, X_temp, X_eda, 
                    label_emotion, label_emotion_ext, label_arousal, label_valence) in enumerate(dataloader): 
        y = label_emotion_ext # 라벨을 변경하고자 하면 이 변수만 바꿔주면 나머지는 y로 적용
        # 예측 오류 계산 
        X_txt, X_wav, X_temp, X_eda, y= X_txt.to(device), X_wav.to(device), X_temp.to(device), X_eda.to(device),y.type(torch.float16).to(device)
        
        X_temp = X_temp.unsqueeze(dim=-1)
        X_eda = X_eda.unsqueeze(dim=-1)
        
        pred = model(X_txt, X_wav, X_temp, X_eda)
        y = F.softmax(y, dim = 1)
        
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.mean().backward() # weighted MSE를 사용할 경우 중간에 sum() or mean()을 넣어줌
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.mean().item(), batch * len(X_txt) # weighted MSE를 사용할 경우 중간에 sum() or mean()을 넣어줌
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        

In [23]:
def test(dataloader, model, loss_fn, mode = 'test'):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    f1 = F1Score(task= 'multiclass', num_classes=7).to(device)   # 바로 multiclassification할 경우
    preds = []
    targets = []
    
    with torch.no_grad():
        # data 순서: text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence
        for batch, (X_txt, X_wav, X_temp, X_eda, 
                        label_emotion, label_emotion_ext, label_arousal, label_valence) in enumerate(dataloader): 
            y = label_emotion_ext # 라벨을 변경하고자 하면 이 변수만 바꿔주면 나머지는 y로 적용
            # 예측 오류 계산
            X_txt, X_wav, X_temp, X_eda, y= X_txt.to(device), X_wav.to(device), X_temp.to(device), X_eda.to(device),y.type(torch.float16).to(device)
            
            X_temp = X_temp.unsqueeze(dim=-1)
            X_eda = X_eda.unsqueeze(dim=-1)
            
            pred = model(X_txt, X_wav, X_temp, X_eda)
            # pred.topk(2)
            preds.append(pred.argmax(1)) # multi regression후 classification으로 변환할 경우
            targets.append(label_emotion) # classification을 할 경우 언제나 사용
            # print('예측라벨분포:',pred[:2], '정답라벨 분포:', label_emotion_ext[:2], '예측정답:', pred.argmax(1)[:2],'정답:', label_emotion[:2])
            print('예측:', pred.argmax(1).tolist()[:15],'\n', '정답:', label_emotion.tolist()[:15])
            # https://discuss.pytorch.org/t/loss-backward-raises-error-grad-can-be-implicitly-created-only-for-scalar-outputs/12152/6
            test_loss += loss_fn(pred, y).mean().item()# weighted MSE를 사용할 경우 중간에 sum() or mean()을 넣어줌 
            
            correct += (pred.argmax(1) == label_emotion.to(device)).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    if mode == 'test':
        print(torch.cat(preds), torch.cat(preds).shape)
        print("f1 score: ", f1(torch.cat(preds).to(device), torch.cat(targets).to(device)))
        print(f"Test Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")
    elif mode == 'val':
        print(f"Validation Error: Accuracy: {(100*correct):>0.1f}%, Avg val loss: {test_loss:>8f} \n")

# 학습시키기

In [24]:
# weigted loss for imbalance data: https://naadispeaks.wordpress.com/2021/07/31/handling-imbalanced-classes-with-weighted-loss-in-pytorch/
# weight 계산
single_emotion = ['angry', 'disgust', 'fear','happy','neutral', 'surprise', 'sad']
total_obs = 0
for i in single_emotion:
    total_obs += Counter(ts_dataset['Emotion'])[i]
print('total obs: ', total_obs)

weight_for_class = []
print(Counter(ts_dataset['Emotion']))
for key, value in sorted(Counter(ts_dataset['Emotion']).items()):
    if key in single_emotion:
        print(f'{key} in single emotion')
        weight_for_class.append(1 - (value/total_obs))
weight_for_class = torch.Tensor(weight_for_class)
weight_for_class

total obs:  12187
Counter({'neutral': 10559, 'happy': 1123, 'happy;neutral': 321, 'surprise': 154, 'angry': 140, 'sad': 108, 'angry;neutral': 76, 'neutral;sad': 69, 'disgust': 61, 'neutral;surprise': 55, 'fear': 42, 'disgust;neutral': 40, 'fear;neutral': 16, 'happy;surprise': 8, 'angry;disgust': 3, 'fear;happy': 2, 'angry;disgust;neutral': 2, 'disgust;neutral;sad': 1, 'angry;disgust;fear;neutral;sad': 1, 'happy;sad': 1, 'happy;neutral;surprise': 1, 'angry;happy;neutral': 1, 'disgust;happy;neutral': 1, 'fear;happy;neutral': 1})
angry in single emotion
disgust in single emotion
fear in single emotion
happy in single emotion
neutral in single emotion
sad in single emotion
surprise in single emotion


tensor([0.9885, 0.9950, 0.9966, 0.9079, 0.1336, 0.9911, 0.9874])

In [25]:
class weighted_MSELoss(nn.Module):
    def __init__(self, weight):
        super().__init__()
        self.weight = weight.to(device)
    def forward(self,inputs,targets):
        return ((inputs - targets)**2) * self.weight

In [26]:
# # 지난 저장한 모델이 있다면
# PATH = './data/test_model.pkl'
# model_tf_mixer = torch.load(PATH)

In [27]:
# loss_fn = nn.CrossEntropyLoss(weight=weight_for_class).to(device)
# loss_fn = nn.CrossEntropyLoss().to(device) # weigth를 주기위해 위의 loss로 임시 변경
loss_fn = weighted_MSELoss(weight = weight_for_class).to(device) # multi target regression(감정별로 count 한 타겟)

In [28]:
lr = 1e-3

# optimizer = optim.SGD(model_tf_mixer.parameters(), lr=lr) # classification
optimizer = optim.Adagrad(model_tf_mixer.parameters(), lr=lr) # regression

## start training mlp fusion mixer

In [29]:
# Set the Training Parameters

epochs = 2
for epoch in range(epochs):
    print(f"---------------Epoch {epoch+1}----------------")
    train(train_dataloader, model_tf_mixer, loss_fn, optimizer)
    test(validation_dataloader, model_tf_mixer, loss_fn, mode = 'val')
print("Done!")

---------------Epoch 1----------------
loss: 0.111285  [    0/ 2607]


In [ ]:
# 실험을 위해 모델 저장
PATH = './data/model_multilabel_MLP.pkl'
torch.save(model_tf_mixer, PATH)

## basic TensorFusionNet 검증

In [203]:
test(test_dataloader, model_tf_mixer, loss_fn, mode = 'test')

/tmp/ipykernel_2139125/2099403102.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(x)


torch.return_types.topk(
values=tensor([[1.0000e+00, 1.0772e-39],
        [1.0000e+00, 3.9076e-26],
        [1.0000e+00, 2.1835e-41],
        [1.0000e+00, 5.0869e-35],
        [1.0000e+00, 1.1395e-17],
        [1.0000e+00, 1.4980e-26],
        [1.0000e+00, 5.1886e-17],
        [1.0000e+00, 4.3641e-25],
        [1.0000e+00, 4.8405e-32],
        [1.0000e+00, 1.9502e-33],
        [1.0000e+00, 2.1790e-29],
        [1.0000e+00, 8.5210e-18],
        [1.0000e+00, 7.0352e-39],
        [1.0000e+00, 9.6373e-39],
        [1.0000e+00, 6.9224e-43],
        [1.0000e+00, 1.9726e-20],
        [1.0000e+00, 3.8851e-37],
        [1.0000e+00, 3.5687e-32],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 3.1350e-26],
        [1.0000e+00, 9.1196e-22],
        [1.0000e+00, 4.2330e-12],
        [1.0000e+00, 2.2743e-42],
        [1.0000e+00, 2.5603e-35],
        [1.0000e+00, 1.5094e-20],
        [1.0000e+00, 2.0666e-22],
        [1.0000e+00, 9.0361e-29],
        [1.0000e+00, 3.2910e-37],
        [1.0000e

RuntimeError: Detected more unique values in `target` than `num_classes`. Expected only 7 but found11 in `target`.

In [381]:
for batch, (X_txt, X_wav, X_temp, X_eda, 
                        label_emotion, label_emotion_ext, label_arousal, label_valence) in list(enumerate(test_dataloader))[:2]:
    pass

{'angry':0, 'disgust':1, 'fear':2,'happy':3,'neutral':4, 'surprise':5, 'sad':6,  'surprise;neutral': 540, 'neutral;sad': 460, 'happy;neutral': 340, 'angry;neutral': 140, 'neutral;disqust': 410, 'neutral;fear': 420, 'happy;surprise': 350, 'happy;angry;neutral': 3040, 'angry;disqust': 10, 'happy;surprise;neutral': 3540, 'happy;fear': 320,'happy;neutral;fear': 3420,'angry;neutral;disqust': 7410, 'neutral;disqust;sad': 4160, 'angry;neutral;disqust;fear;sad': 10000,'happy;sad': 360, 'happy;neutral;disqust': 3410}

In [396]:
# X_temp = X_temp.unsqueeze(dim=-1)
# X_eda = X_eda.unsqueeze(dim=-1)

probs = model_tf_mixer(X_txt.to(device), X_wav.to(device), X_temp.to(device), X_eda.to(device))

values = probs.topk(2)
# print(values)
diffs = abs(torch.diff(values.values))
for idx,diff in zip(values.indices, diffs):
    if diff >= 0.1:
        sorted, idx = torch.sort(idx)
        # print(sorted)
        print(torch.stack([100*sorted[0] + 10*sorted[1]], dim=0))
    else:
        print(idx[0])

    

tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([10], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([10], device='cuda:0')
tensor([10], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50], device='cuda:0')
tensor([50

/tmp/ipykernel_2139125/2099403102.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(x)
